다항분포 나이브 베이즈를 활용한 영화 리뷰 분류  
영화 리뷰에 다항분포 나이브 베이즈를 활용해 영화 리뷰가 긍정적인지 부정적인지 분류한다.

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# 다항분포 나이브 베이즈를 사용하기 위해 import 한다.
from sklearn.naive_bayes import MultinomialNB
# 데이터에 출현한 모든 단어 개수 만큼의 크기를 가진 벡터를 만들고 고정된 벡터로 표현하기 위해 import 한다.
from sklearn.feature_extraction.text import CountVectorizer

데이터 획득  
영화 리뷰에 다항분포 나이브 베이즈를 활용해 영화 리뷰가 긍정적인지 부정적인지 분류한다.  
movie_review: 영화 리뷰, type: 리뷰가 긍정적(positive)인지 부정적(negative)인지 나타낸다.

In [3]:
# 학습 데이터 준비
review_list = [
    {'movie_review' : 'this is great great movie. I will watch again', 'type' : 'positive'},
    {'movie_review' : 'I like this movie', 'type' : 'positive'},
    {'movie_review' : 'amazing movie in this year', 'type' : 'positive'},
    {'movie_review' : 'cool my boyfriend also said the movie is cool', 'type' : 'positive'},
    {'movie_review' : 'awesome of the awesome movie ever', 'type' : 'positive'},
    {'movie_review' : 'shame I wasted money and time', 'type' : 'negative'},
    {'movie_review' : 'regret on this move. I will never never what movie from this director', 'type' : 'negative'},
    {'movie_review' : 'I do not like this movie', 'type' : 'negative'},
    {'movie_review' : 'I do not like actors in this movie', 'type' : 'negative'},
    {'movie_review' : 'boring boring sleeping movie', 'type' : 'negative'}
]
df = pd.DataFrame(review_list)
df

,movie_review,type
0,this is great great movie. I will watch again,positive
1,I like this movie,positive
2,amazing movie in this year,positive
3,cool my boyfriend also said the movie is cool,positive
4,awesome of the awesome movie ever,positive
5,shame I wasted money and time,negative
6,regret on this move. I will never never what m...,negative
7,I do not like this movie,negative
8,I do not like actors in this movie,negative
9,boring boring sleeping movie,negative


In [4]:
# 학습 데이터 다듬기
# 사이킷런의 다항분포 나이브 베이즈 분류기는 숫자 데이터만 다루기 때문에 positive와 negative를 1과 0으로 치환한다.
df['label'] = df.type.map({'positive': 1, 'negative': 0})
df

,movie_review,type,label
0,this is great great movie. I will watch again,positive,1
1,I like this movie,positive,1
2,amazing movie in this year,positive,1
3,cool my boyfriend also said the movie is cool,positive,1
4,awesome of the awesome movie ever,positive,1
5,shame I wasted money and time,negative,0
6,regret on this move. I will never never what m...,negative,0
7,I do not like this movie,negative,0
8,I do not like actors in this movie,negative,0
9,boring boring sleeping movie,negative,0


학습에 사용할 데이터와 레이블(답, 실제값, 타겟, 클래스)로 분리한다.

In [5]:
df_x = df['movie_review'] # 피쳐
df_x

0        this is great great movie. I will watch again
1                                    I like this movie
2                           amazing movie in this year
3        cool my boyfriend also said the movie is cool
4                    awesome of the awesome movie ever
5                        shame I wasted money and time
6    regret on this move. I will never never what m...
7                             I do not like this movie
8                   I do not like actors in this movie
9                         boring boring sleeping movie
Name: movie_review, dtype: object

In [6]:
df_y = df['label'] # 타겟
df_y

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

영화 리뷰(movie_review)으로 학습을 진행하고 레이블은 label을 사용해서 긍정적, 부정적 여부를 판단한다.

다항분포 나이브 베이즈의 입력 데이터는 고정된(동일한) 크기의 벡터이어야 한다.

In [7]:
# CountVectorizer 객체는 문자열에 출현한 단어를 오름차순으로 정렬해 단어의 위치로 행렬을 만들어 리턴한다.
# 베르누이 나이브 베이즈는 같은 단어가 여러번 출현하더라도 무조건 1을 리턴해야 하기 때문에 binary=True 옵션을 지정하지만
# 다항분포 나이브 베이즈는 같은 단어가 여러번 출현하면 출현한 빈도수를 나타내야 하기 때문에 binary=True 옵션을 생략한다.
# cv = CountVectorizer(binary=True) # 베르누이 나이브 베이즈
cv = CountVectorizer() # 다항분포 나이브 베이즈
x_train = cv.fit_transform(df_x) # 학습 후 적용
encoding_input = x_train.toarray() # 넘파이 타입의 배열로 데이터를 변환한다.
print(type(encoding_input))
print(encoding_input)

<class 'numpy.ndarray'>
[[0 1 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  1]
 [0 0 1 0 0 0 0 1 2 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
  0]
 [0 0 0 0 0 2 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0
  0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 2 0 0 1 1 0 0 0 0 2 0 0 0 1 1
  0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
  0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
  0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0]]


위의 행렬에서 볼 수 있듯이 영화 리뷰에서 총 37개의 단어가 발견되어 각 영화 리뷰가 37개 크기의 벡터로 표현(인코딩)된 것을 확인할 수 있다.  
다항분포 나이브 베이즈에서 사용하기 위해 영화 리뷰에 중복되서 출현한 단어있으면 출현한 횟수로 표현된 것을 알 수 있다.

In [8]:
for s in cv.inverse_transform(encoding_input):
    print(s)

['again' 'great' 'is' 'movie' 'this' 'watch' 'will']
['like' 'movie' 'this']
['amazing' 'in' 'movie' 'this' 'year']
['also' 'boyfriend' 'cool' 'is' 'movie' 'my' 'said' 'the']
['awesome' 'ever' 'movie' 'of' 'the']
['and' 'money' 'shame' 'time' 'wasted']
['director' 'from' 'move' 'movie' 'never' 'on' 'regret' 'this' 'what'
 'will']
['do' 'like' 'movie' 'not' 'this']
['actors' 'do' 'in' 'like' 'movie' 'not' 'this']
['boring' 'movie' 'sleeping']


In [9]:
print(cv.get_feature_names())

['actors', 'again', 'also', 'amazing', 'and', 'awesome', 'boring', 'boyfriend', 'cool', 'director', 'do', 'ever', 'from', 'great', 'in', 'is', 'like', 'money', 'move', 'movie', 'my', 'never', 'not', 'of', 'on', 'regret', 'said', 'shame', 'sleeping', 'the', 'this', 'time', 'wasted', 'watch', 'what', 'will', 'year']


다항분포 나이브 베이즈 모델 학습하기  
사이킷런의 다항분포 나이브 베이즈 분류기는 기본적으로 라플라스 스무딩을 지원하므로 학습 데이터에 없던 단어가 테스트 데이터에 있어도 분류가 잘 진행된다.

In [10]:
df_y

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

In [11]:
mnb = MultinomialNB() # 다항분포 나이브 베이즈 모델을 만든다.
mnb.fit(x_train, df_y) # 다항분포 나이브 베이즈 모델을 학습시킨다.

MultinomialNB()

테스트 데이터를 다듬고 테스트 실행

In [12]:
test_feedback_list = [
    {'movie_review': 'great great great movie ever', 'type': 'positive'},
    {'movie_review': 'I like this amazing movie', 'type': 'positive'},
    {'movie_review': 'my boyfriend said great movie ever', 'type': 'positive'},
    {'movie_review': 'cool cool cool', 'type': 'positive'},
    {'movie_review': 'awesome boyfriend said cool movie ever', 'type': 'positive'},
    {'movie_review': 'shame shame shame', 'type': 'negative'},
    {'movie_review': 'awesome director shame movie boring movie', 'type': 'negative'},
    {'movie_review': 'do not like this movie', 'type': 'negative'},
    {'movie_review': 'I do not like this boring movie', 'type': 'negative'},
    {'movie_review': 'aweful terrible boring movie', 'type': 'negative'}
]
test_df = pd.DataFrame(test_feedback_list)
test_df['label'] = test_df.type.map({'positive': 1, 'negative': 0})
test_df

,movie_review,type,label
0,great great great movie ever,positive,1
1,I like this amazing movie,positive,1
2,my boyfriend said great movie ever,positive,1
3,cool cool cool,positive,1
4,awesome boyfriend said cool movie ever,positive,1
5,shame shame shame,negative,0
6,awesome director shame movie boring movie,negative,0
7,do not like this movie,negative,0
8,I do not like this boring movie,negative,0
9,aweful terrible boring movie,negative,0


In [13]:
test_x = test_df['movie_review']
test_x

0                 great great great movie ever
1                    I like this amazing movie
2           my boyfriend said great movie ever
3                               cool cool cool
4       awesome boyfriend said cool movie ever
5                            shame shame shame
6    awesome director shame movie boring movie
7                       do not like this movie
8              I do not like this boring movie
9                 aweful terrible boring movie
Name: movie_review, dtype: object

In [14]:
test_y = test_df['label']
test_y

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

In [15]:
# CountVectorizer 객체는 학습 데이터를 다듬을 때 이미 학습을 시켰으므로 테스트 시에는 적용만 시키면 된다.
x_test = cv.transform(test_x) # 적용

모델 테스트

In [17]:
predict = mnb.predict(x_test) # 학습 데이터로 학습한 결과에 따른 테스트 데이터의 예측값을 계산한다.
accuracy = accuracy_score(test_y, predict) #accuracy_score(실제값, 예측값)
print('정확도: {:.2%}'.format(accuracy))

정확도: 100.00%


In [18]:
conf_matrix = confusion_matrix(test_y, predict)
print(conf_matrix)

[[5 0]
 [0 5]]


In [19]:
print(classification_report(test_y, predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [20]:
comparison = pd.DataFrame({'실제값': test_y, '예측값': predict})
comparison

,실제값,예측값
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0
